### Project categories

In [ ]:
from path import Path
import fastai.vision.all as V

import data

path = Path('images')
categories = 'la-pavoni-europiccola', 'niche-zero', 'moccamaster'

### Download data

In [ ]:
data.create_dataset(path, categories)
data.remove_failed_images(path)

### Create dataset

In [ ]:
data_block = V.DataBlock(
    blocks=(V.ImageBlock, V.CategoryBlock),
    get_items=V.get_image_files,
    splitter=V.RandomSplitter(valid_pct=0.2, seed=42),
    item_tfms=V.Resize(480),  # Presizing for single interpolation on GPU
    batch_tfms=V.aug_transforms(size=224, min_scale=0.75),
    get_y=V.parent_label,
)

### Create dataloaders

In [ ]:
dls = data_block.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

### Create model using transfer learning

In [ ]:
learn = V.cnn_learner(dls, V.resnet18, metrics=V.error_rate)

### Fine tune model

In [ ]:
learn.fine_tune(1)

### Confusion Matrix 

In [ ]:
interp = V.ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

### Top Losses

In [ ]:
interp.plot_top_losses(2, nrows=2)